In [1]:
#!pip install roboflow
import os
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
import numpy as np
from roboflow import Roboflow

# Define the folder containing images
images_folder = 'data_2/TEST/8'

# Create a list to store predictions
all_predictions = []

def crop_images(images_folder, padding=10):
    rf = Roboflow(api_key="5QKKlp6IjU5UTbSv406u")
    project = rf.workspace("is4242-gmgjf").project("road-sign-image-detection")
    version = project.version(2)
    dataset = version.download("yolokeras")

    model = version.model
    # Optionally, change the confidence and overlap thresholds (values are percentages)
    model.confidence = 60
    model.overlap = 35

    cropped_images = []

    # Loop through the images folder
    for image_file in os.listdir(images_folder):
        if image_file.endswith('.jpg') or image_file.endswith('.jpeg') or image_file.endswith('.png'):
            # Load the image
            image_path = os.path.join(images_folder, image_file)
            image = mpimg.imread(image_path)
            prediction = model.predict(image_path)

            # Extract predictions as JSON
            predictions_json = prediction.json()
            
            # Crop the image to include some extra space around the bounding box
            for bounding_box in predictions_json['predictions']:
                x = bounding_box['x']
                y = bounding_box['y']
                width = bounding_box['width']
                height = bounding_box['height']

                # Calculate corner points with extra padding
                x1 = int(x - (width / 2) - padding)
                y1 = int(y - (height / 2) - padding)
                x2 = int(x + (width / 2) + padding)
                y2 = int(y + (height / 2) + padding)

                # Ensure the cropped area is within the image bounds
                x1 = max(x1, 0)
                y1 = max(y1, 0)
                x2 = min(x2, image.shape[1])
                y2 = min(y2, image.shape[0])

                # Crop the image
                cropped_image = image[y1:y2, x1:x2]

                # Append the cropped image to the list
                cropped_images.append(cropped_image)

                # Plot the cropped image
                plt.imshow(cropped_image)
                plt.axis('off')
                plt.show()
                print(prediction.plot())

    # Convert the list of cropped images to a numpy array
    cropped_images_array = np.array(cropped_images)
    
    return cropped_images_array

# Example usage:
# cropped_images_array = crop_images('images_folder_path', model)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Road-Sign-Image-Detection-2 in yolokeras:: 100%|██████████| 378/378 [00:00<00:00, 5899.14it/s]
